In [ ]:
!pip install tabpfn shap eli5 lime torch torchvision torchaudio transformers datasets evaluate wandb mlflow optuna scikit-learn pandas numpy matplotlib seaborn plotly jupyterlab jupyterlab-git jupyterlab-lsp black jupyter-black autopep8 isort pylint flake8 pre-commit nbdev